In [1]:
import scanpy as sc
from muon import atac as ac
import numpy as np
from scipy.sparse import csr_matrix
import functools

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata_rna = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_srt_rna.h5ad")
adata_atac = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_srt_peaks.h5ad")
adata_activity = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_srt_activity.h5ad")

In [3]:
adata_rna.layers["raw_counts"] = adata_rna.X.copy()

In [4]:
adata_atac.obs = adata_rna.obs
adata_activity.obs = adata_rna.obs

In [5]:
adata_src = adata_rna[adata_rna.obs["orig.ident"]=="GLI3_WT_A4_day19"]
adata_tgt_all = adata_atac[adata_atac.obs["orig.ident"].isin(("GLI3_WT_A4_day19", "GLI3_KO_D3_day19"))]

In [6]:
sc.pp.normalize_total(adata_src)
sc.pp.log1p(adata_src)

ac.pp.tfidf(adata_tgt_all)



/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/muon/_atac/preproc.py:84: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [7]:
adata_tgt_all.X.min()

0.0

In [9]:
sc.pp.highly_variable_genes(adata_src, n_top_genes=2000, subset=True)
sc.pp.highly_variable_genes(adata_tgt_all, n_top_genes=100000, subset=True)
sc.pp.highly_variable_genes(adata_tgt_all, n_top_genes=50, subset=False)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning

In [11]:
adata_tgt_all.var["highly_variable"].sum()

50

In [12]:
adata_tgt = adata_tgt_all[adata_tgt_all.obs["orig.ident"]=="GLI3_KO_D3_day19"]
adata_src_atac = adata_tgt_all[adata_tgt_all.obs["orig.ident"]=="GLI3_WT_A4_day19"]

In [13]:
rng = np.random.default_rng(1)
n_cells_source = adata_src.n_obs

n_samples_train_source = int(n_cells_source * 0.6)
n_samples_test_source = n_cells_source - n_cells_source

inds_train_source = np.asarray(rng.choice(n_cells_source, (n_samples_train_source,), replace=False))
inds_test_source = list(set(list(range(n_cells_source))) - set(np.asarray(inds_train_source)))

In [14]:
rng = np.random.default_rng(0)
n_cells_target = adata_tgt.n_obs

n_samples_train_target = int(n_cells_target * 0.6)
n_samples_test_target = n_cells_target - n_cells_target

inds_train_target = np.asarray(rng.choice(n_cells_target, (n_samples_train_target,), replace=False))
inds_test_target = list(set(list(range(n_cells_target))) - set(np.asarray(inds_train_target)))

In [15]:
adata_src_train = adata_src[inds_train_source]
adata_tgt_train = adata_tgt[inds_train_target]

adata_src_test = adata_src[inds_test_source]
adata_tgt_test = adata_tgt[inds_test_target]

adata_src_atac_train = adata_src_atac[inds_train_source]
adata_src_atac_test = adata_src_atac[inds_test_source]

In [16]:
adata_activity_red = adata_activity[adata_activity.obs["orig.ident"]=="GLI3_KO_D3_day19"]

adata_activity_train = adata_activity_red[inds_train_target]
adata_activity_test = adata_activity_red[inds_test_target]

In [17]:
assert (adata_activity_train.obs_names == adata_tgt_train.obs_names).all()
assert (adata_activity_test.obs_names == adata_tgt_test.obs_names).all()

In [18]:
adata_src_train.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_train_processed.h5ad")
adata_src_test.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_test_processed.h5ad")
adata_tgt_train.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_train_processed.h5ad")
adata_tgt_test.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_test_processed.h5ad")
adata_activity_train.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_activity_train_processed.h5ad")
adata_activity_test.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_activity_test_processed.h5ad")
adata_src_atac_train.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_atac_train_processed.h5ad")
adata_src_atac_test.write("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_arc_atac_test_processed.h5ad")